In [33]:
import pandas as pd
import numpy as np
import joblib
import plotly.express as px
import os
import polars as pl
import glob
from IPython.display import display, HTML
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix

In [30]:
# Load the skaters dataset
skaters_df = pd.read_excel("../DataCleaning/Instat_DataCleaning/SkatersCleaned.xlsx")

# Display the DataFrame as a scrollable HTML table
display(HTML(skaters_df.to_html(index=False, max_rows=5, max_cols=None)))

Shirt number,Player,Position,Games played,All shifts,Goals,First assist,Second assist,Assists,Points,+/-,Scoring chances,Penalties drawn,Faceoffs,Faceoffs won,"Faceoffs won, %",Hits,Shots,Shots on goal,Blocked shots,Power play shots,Short-handed shots,Passes to the slot,Faceoffs in DZ,Faceoffs won in DZ,"Faceoffs won in DZ, %",Faceoffs in NZ,Faceoffs won in NZ,"Faceoffs won in NZ, %",Faceoffs in OZ,Faceoffs won in OZ,"Faceoffs won in OZ, %",Puck touches,Plus,Minus,Penalties,Faceoffs lost,Hits against,Error leading to goal,Dump ins,Dump outs,Team goals when on ice,Opponent's goals when on ice,Power play,Successful power play,Short-handed,Penalty killing,Missed shots,% shots on goal,Slapshot,Wrist shot,Shootouts,Shootouts scored,Shootouts missed,1-on-1 shots,1-on-1 goals,"Shots conversion 1 on 1, %",Positional attack shots,Shots 5 v 5,Counter-attack shots,xG per shot,xG (Expected goals),xG per goal,Net xG (xG player on - opp. team's xG),Team xG when on ice,Opponent's xG when on ice,xG conversion,CORSI,CORSI-,CORSI+,"CORSI for, %",Fenwick for,Fenwick against,"Fenwick for, %",Puck battles,Puck battles won,"Puck battles won, %",Puck battles in DZ,Puck battles in NZ,Puck battles in OZ,Shots blocking,Dekes,Dekes successful,Dekes unsuccessful,"Dekes successful, %",Passes,Accurate passes,"Accurate passes, %",Pre-shots passes,Pass receptions,Scoring chances - total,Scoring chances - scored,Scoring chances missed,Scoring chances saved,"Scoring Chances, %",Inner slot shots - total,Inner slot shots - scored,Inner slot shots - missed,Inner slot shots - saved,"Inner slot shots, %",Outer slot shots - total,Outer slot shots - scored,Outer slot shots - missed,Outer slot shots - saved,"Outer slot shots, %",Blocked shots from the slot,Blocked shots outside of the slot,Takeaways,Puck retrievals after shots,Opponent’s dump-in retrievals,Takeaways in DZ,Loose puck recovery,Takeaways in NZ,Takeaways in OZ,EV DZ retrievals,Puck losses,Puck losses in DZ,EV OZ retrievals,Puck losses in NZ,Power play retrievals,Penalty kill retrievals,Puck losses in OZ,Entries,Entries via pass,Entries via dump in,Entries via stickhandling,Breakouts,Breakouts via pass,Breakouts via dump out,Breakouts via stickhandling,National team,Height,Weight,Contract,Active hand,Type,Time on ice_minutes,Time on ice_seconds,Penalty time_minutes,Penalty time_seconds,Puck control time_minutes,Puck control time_seconds,Power play time_minutes,Power play time_seconds,Short-handed time_minutes,Short-handed time_seconds,Playing in attack_minutes,Playing in attack_seconds,Playing in defense_minutes,Playing in defense_seconds,OZ possession_minutes,OZ possession_seconds,NZ possession_minutes,NZ possession_seconds,DZ possession_minutes,DZ possession_seconds
6,Alyssa Wilson,F,23,17.0,0.04,0.04,0.04,0.09,0.13,-1.04,0.04,0.04,0.09,0.04,0.50,0.13,2.00,0.74,0.61,0.00,0.0,0.17,0.04,0.04,1.00,0.04,0.00,0.00,0.0,0.00,0.00,57.0,0.22,1.26,0.17,0.04,0.04,0.96,1.61,1.96,0.22,1.26,0.00,0.00,0.00,0.00,0.65,0.37,0.74,0.87,0.0,0.0,0.0,0.00,0.0,0.0,1.17,2.00,0.04,0.31,0.05,0.29,-0.75,0.46,1.22,0.0091,-8.0,18,10.0,0.35,7.0,14.0,0.35,10.0,5.2,0.54,7.0,1.61,1.39,0.78,0.26,0.22,0.04,0.83,12.0,10.0,0.83,0.22,6.0,0.26,0.04,0.04,0.17,0.17,0.04,0.04,0.00,0.00,1.0,0.22,0.0,0.04,0.17,0.0,0.00,0.61,7.0,1.87,1.09,5.20,1.48,1.61,0.39,1.39,4.20,1.30,0.48,1.04,0.00,0.00,1.87,1.48,0.52,0.22,0.74,2.30,1.48,0.13,0.70,0,0,0.0,0,Right,Even Strength,15,20,0,26,0,16,0,0,0,0,6,15,7,7,4,56,3,8,7,16
12,Ella Bynan,F,24,16.0,0.00,0.08,0.13,0.21,0.21,-0.58,0.38,0.13,9.00,4.30,0.48,0.00,1.54,1.17,0.21,0.00,0.0,0.25,3.80,1.79,0.47,2.80,1.38,0.49,2.4,1.17,0.48,55.0,0.21,0.79,0.04,4.70,0.04,0.08,1.79,0.50,0.21,0.79,0.00,0.00,0.00,0.00,0.17,0.76,0.54,0.38,0.0,0.0,0.0,0.00,0.0,0.0,0.50,1.54,0.08,1.33,0.12,0.00,-0.60,0.43,1.03,NaN,-7.0,17,9.0,0.36,7.0,13.0,0.35,14.0,5.9,0.42,6.0,2.00,5.70,0.92,0.63,0.42,0.21,0.67,9.0,7.0,0.71,0.29,8.0,0.71,0.00,0.08,0.63,0.00,0.38,0.00,0.04,0.33,0.0,0.33,0.0,0.04,0.29,0.0,0.00,0.21,3.4,1.04,0.04,2.50,0.21,0.67,0.

In [31]:
# Drop unneeded columns
skaters_df.drop(['Player', 'Contract'], axis=1, inplace=True)

# Find remaining categorical columns and one‐hot encode them
cat_cols = skaters_df.select_dtypes(include=['object']).columns.tolist()
skaters_df = pd.get_dummies(skaters_df, columns=cat_cols)

display(HTML(skaters_df.to_html(index=False, max_rows=5, max_cols=None)))

Shirt number,Games played,All shifts,Goals,First assist,Second assist,Assists,Points,+/-,Scoring chances,Penalties drawn,Faceoffs,Faceoffs won,"Faceoffs won, %",Hits,Shots,Shots on goal,Blocked shots,Power play shots,Short-handed shots,Passes to the slot,Faceoffs in DZ,Faceoffs won in DZ,"Faceoffs won in DZ, %",Faceoffs in NZ,Faceoffs won in NZ,"Faceoffs won in NZ, %",Faceoffs in OZ,Faceoffs won in OZ,"Faceoffs won in OZ, %",Puck touches,Plus,Minus,Penalties,Faceoffs lost,Hits against,Error leading to goal,Dump ins,Dump outs,Team goals when on ice,Opponent's goals when on ice,Power play,Successful power play,Short-handed,Penalty killing,Missed shots,% shots on goal,Slapshot,Wrist shot,Shootouts,Shootouts scored,Shootouts missed,1-on-1 shots,1-on-1 goals,"Shots conversion 1 on 1, %",Positional attack shots,Shots 5 v 5,Counter-attack shots,xG per shot,xG (Expected goals),xG per goal,Net xG (xG player on - opp. team's xG),Team xG when on ice,Opponent's xG when on ice,xG conversion,CORSI,CORSI-,CORSI+,"CORSI for, %",Fenwick for,Fenwick against,"Fenwick for, %",Puck battles,Puck battles won,"Puck battles won, %",Puck battles in DZ,Puck battles in NZ,Puck battles in OZ,Shots blocking,Dekes,Dekes successful,Dekes unsuccessful,"Dekes successful, %",Passes,Accurate passes,"Accurate passes, %",Pre-shots passes,Pass receptions,Scoring chances - total,Scoring chances - scored,Scoring chances missed,Scoring chances saved,"Scoring Chances, %",Inner slot shots - total,Inner slot shots - scored,Inner slot shots - missed,Inner slot shots - saved,"Inner slot shots, %",Outer slot shots - total,Outer slot shots - scored,Outer slot shots - missed,Outer slot shots - saved,"Outer slot shots, %",Blocked shots from the slot,Blocked shots outside of the slot,Takeaways,Puck retrievals after shots,Opponent’s dump-in retrievals,Takeaways in DZ,Loose puck recovery,Takeaways in NZ,Takeaways in OZ,EV DZ retrievals,Puck losses,Puck losses in DZ,EV OZ retrievals,Puck losses in NZ,Power play retrievals,Penalty kill retrievals,Puck losses in OZ,Entries,Entries via pass,Entries via dump in,Entries via stickhandling,Breakouts,Breakouts via pass,Breakouts via dump out,Breakouts via stickhandling,National team,Height,Weight,Time on ice_minutes,Time on ice_seconds,Penalty time_minutes,Penalty time_seconds,Puck control time_minutes,Puck control time_seconds,Power play time_minutes,Power play time_seconds,Short-handed time_minutes,Short-handed time_seconds,Playing in attack_minutes,Playing in attack_seconds,Playing in defense_minutes,Playing in defense_seconds,OZ possession_minutes,OZ possession_seconds,NZ possession_minutes,NZ possession_seconds,DZ possession_minutes,DZ possession_seconds,Position_D,Position_F,Active hand_Left,Active hand_Right,Type_Even Strength,Type_Penalty Kill,Type_Power Play,Type_Total
6,23,17.0,0.04,0.04,0.04,0.09,0.13,-1.04,0.04,0.04,0.09,0.04,0.50,0.13,2.00,0.74,0.61,0.00,0.0,0.17,0.04,0.04,1.00,0.04,0.00,0.00,0.0,0.00,0.00,57.0,0.22,1.26,0.17,0.04,0.04,0.96,1.61,1.96,0.22,1.26,0.00,0.00,0.00,0.00,0.65,0.37,0.74,0.87,0.0,0.0,0.0,0.00,0.0,0.0,1.17,2.00,0.04,0.31,0.05,0.29,-0.75,0.46,1.22,0.0091,-8.0,18,10.0,0.35,7.0,14.0,0.35,10.0,5.2,0.54,7.0,1.61,1.39,0.78,0.26,0.22,0.04,0.83,12.0,10.0,0.83,0.22,6.0,0.26,0.04,0.04,0.17,0.17,0.04,0.04,0.00,0.00,1.0,0.22,0.0,0.04,0.17,0.0,0.00,0.61,7.0,1.87,1.09,5.20,1.48,1.61,0.39,1.39,4.20,1.30,0.48,1.04,0.00,0.00,1.87,1.48,0.52,0.22,0.74,2.30,1.48,0.13,0.70,0,0,0.0,15,20,0,26,0,16,0,0,0,0,6,15,7,7,4,56,3,8,7,16,False,True,False,True,True,False,False,False
12,24,16.0,0.00,0.08,0.13,0.21,0.21,-0.58,0.38,0.13,9.00,4.30,0.48,0.00,1.54,1.17,0.21,0.00,0.0,0.25,3.80,1.79,0.47,2.80,1.38,0.49,2.4,1.17,0.48,55.0,0.21,0.79,0.04,4.70,0.04,0.08,1.79,0.50,0.21,0.79,0.00,0.00,0.00,0.00,0.17,0.76,0.54,0.38,0.0,0.0,0.0,0.00,0.0,0.0,0.50,1.54,0.08,1.33,0.12,0.00,-0.60,0.43,1.03,NaN,-7.0,17,9.0,0.36,7.0,13.0,0.35,14.0,5.9,0.42,6.0,2.00,5.70,0.92,0.63,0.42,0.21,0.67,9.0,7.0,0.71,0.29,8.0,0.71,0.00,0.08,0.63,0.00,0.38,0.00,0.04,

In [34]:
# Prepare data for linear regression
# Define target and columns to drop
target = "Goals"
drop_cols = [
    target,
    "Points",
    "Scoring chances",
    "Shots conversion 1 on 1, %",
    "Inner slot shots - scored",
    "Outer slot shots - scored",
    "xG per goal",
    "+/-",
    "Team goals when on ice",
    "Plus"
]

# Features and target
X = skaters_df.drop(columns=drop_cols, errors="ignore")
y = skaters_df[target]

# Set up cross‐validation and model
kf = KFold(n_splits=5, shuffle=True, random_state=42)
model = LinearRegression()

# Evaluate with MSE
mse_scores = -cross_val_score(model, X, y, cv=kf, scoring="neg_mean_squared_error")
print(f"CV mean MSE: {mse_scores.mean():.4f}")

# Train final model on all data
model.fit(X, y)
print("Coefficients:", dict(zip(X.columns, model.coef_)))

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "c:\repos\Hood-Hockey\venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\repos\Hood-Hockey\venv\Lib\site-packages\sklearn\base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\repos\Hood-Hockey\venv\Lib\site-packages\sklearn\linear_model\_base.py", line 601, in fit
    X, y = validate_data(
           ~~~~~~~~~~~~~^
        self,
        ^^^^^
    ...<5 lines>...
        force_writeable=True,
        ^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "c:\re